# Feature Engineering

## Combine Data

In [1]:
import pandas as pd

# Load datasets
abm = pd.read_csv("abm.csv")
card = pd.read_csv("card.csv")
cheque = pd.read_csv("cheque.csv")
eft = pd.read_csv("eft.csv")
emt = pd.read_csv("emt.csv")
wire = pd.read_csv("wire.csv")
kyc = pd.read_csv("kyc.csv")
kyc_industry_codes = pd.read_csv("kyc_industry_codes.csv")

In [2]:
abm.head()

,abm_id,customer_id,amount_cad,debit_credit,cash_indicator,country,province,city,transaction_date,transaction_time
0,ABM00000000000000006,SYNCID0000000014,25.41,credit,False,CA,SK,REGINA,2022-11-16,17:37:41
1,ABM00000000000000008,SYNCID0000000034,238.17,debit,True,CA,ON,TORONTO,2022-11-18,10:22:59
2,ABM00000000000000009,SYNCID0000000034,1655.43,credit,False,CA,NaN,other,2022-12-29,11:56:08
3,ABM00000000000000010,SYNCID0000000034,620.69,credit,True,CA,NaN,other,2023-01-22,16:48:12
4,ABM00000000000000011,SYNCID0000000034,323.70,debit,True,CA,NaN,other,2022-11-14,13:24:45


In [4]:
# Standardizing column names for merging
abm["source"] = "abm"
abm = abm.rename(columns={"abm_id": "transaction_id"})

card["source"] = "card"
card = card.rename(columns={"card_trxn_id": "transaction_id"})

cheque["source"] = "cheque"
cheque = cheque.rename(columns={"cheque_id": "transaction_id"})

eft["source"] = "eft"
eft = eft.rename(columns={"eft_id": "transaction_id"})

emt["source"] = "emt"
emt = emt.rename(columns={"emt_id": "transaction_id"})

wire["source"] = "wire"
wire = wire.rename(columns={"wire_id": "transaction_id"})

In [5]:
# Standardizing missing columns across datasets
for df in [abm, card, cheque, eft, emt, wire]:
    df["country"] = df.get("country", pd.NA)
    df["province"] = df.get("province", pd.NA)
    df["city"] = df.get("city", pd.NA)
    df["transaction_time"] = df.get("transaction_time", pd.NA)

In [6]:
# Union all transaction data
transactions = pd.concat([abm, card, cheque, eft, emt, wire], ignore_index=True)

In [7]:
transactions.head()

,transaction_id,customer_id,amount_cad,debit_credit,cash_indicator,country,province,city,transaction_date,transaction_time,source,merchant_category,ecommerce_ind
0,ABM00000000000000006,SYNCID0000000014,25.41,credit,False,CA,SK,REGINA,2022-11-16,17:37:41,abm,NaN,NaN
1,ABM00000000000000008,SYNCID0000000034,238.17,debit,True,CA,ON,TORONTO,2022-11-18,10:22:59,abm,NaN,NaN
2,ABM00000000000000009,SYNCID0000000034,1655.43,credit,False,CA,NaN,other,2022-12-29,11:56:08,abm,NaN,NaN
3,ABM00000000000000010,SYNCID0000000034,620.69,credit,True,CA,NaN,other,2023-01-22,16:48:12,abm,NaN,NaN
4,ABM00000000000000011,SYNCID0000000034,323.70,debit,True,CA,NaN,other,2022-11-14,13:24:45,abm,NaN,NaN


In [8]:
transactions.shape

(1494097, 13)

In [ ]:
transactions.to_csv("transactions.csv", index=False)

In [14]:
# Left join with kyc on customer_id
merged_data = transactions.merge(kyc, on="customer_id", how="left")

In [15]:
merged_data.head()

,transaction_id,customer_id,amount_cad,debit_credit,cash_indicator,country_x,province_x,city_x,transaction_date,transaction_time,...,merchant_category,ecommerce_ind,country_y,province_y,city_y,industry_code,employee_count,sales,established_date,onboard_date
0,ABM00000000000000006,SYNCID0000000014,25.41,credit,False,CA,SK,REGINA,2022-11-16,17:37:41,...,NaN,NaN,CA,ON,MARKHAM,4275,0.0,NaN,2016-09-14,NaN
1,ABM00000000000000008,SYNCID0000000034,238.17,debit,True,CA,ON,TORONTO,2022-11-18,10:22:59,...,NaN,NaN,CA,ON,LONDON,0911,NaN,120311.0,2010-09-13,2012-04-17
2,ABM00000000000000009,SYNCID0000000034,1655.43,credit,False,CA,NaN,other,2022-12-29,11:56:08,...,NaN,NaN,CA,ON,LONDON,0911,NaN,120311.0,2010-09-13,2012-04-17
3,ABM00000000000000010,SYNCID0000000034,620.69,credit,True,CA,NaN,other,2023-01-22,16:48:12,...,NaN,NaN,CA,ON,LONDON,0911,NaN,120311.0,2010-09-13,2012-04-17
4,ABM00000000000000011,SYNCID0000000034,323.70,debit,True,CA,NaN,other,2022-11-14,13:24:45,...,NaN,NaN,CA,ON,LONDON,0911,NaN,120311.0,2010-09-13,2012-04-17


In [16]:
merged_data.shape

(1494097, 21)

In [19]:
# Ensure industry_code is of type object (string) before merging
merged_data["industry_code"] = merged_data["industry_code"].astype(str)
kyc_industry_codes["industry_code"] = kyc_industry_codes["industry_code"].astype(str)

# Left join with kyc_industry_codes on industry_code
final_data = merged_data.merge(kyc_industry_codes, on="industry_code", how="left")

In [20]:
final_data.head()

,transaction_id,customer_id,amount_cad,debit_credit,cash_indicator,country_x,province_x,city_x,transaction_date,transaction_time,...,ecommerce_ind,country_y,province_y,city_y,industry_code,employee_count,sales,established_date,onboard_date,industry
0,ABM00000000000000006,SYNCID0000000014,25.41,credit,False,CA,SK,REGINA,2022-11-16,17:37:41,...,NaN,CA,ON,MARKHAM,4275,0.0,NaN,2016-09-14,NaN,Painting and Decorating Work
1,ABM00000000000000008,SYNCID0000000034,238.17,debit,True,CA,ON,TORONTO,2022-11-18,10:22:59,...,NaN,CA,ON,LONDON,0911,NaN,120311.0,2010-09-13,2012-04-17,NaN
2,ABM00000000000000009,SYNCID0000000034,1655.43,credit,False,CA,NaN,other,2022-12-29,11:56:08,...,NaN,CA,ON,LONDON,0911,NaN,120311.0,2010-09-13,2012-04-17,NaN
3,ABM00000000000000010,SYNCID0000000034,620.69,credit,True,CA,NaN,other,2023-01-22,16:48:12,...,NaN,CA,ON,LONDON,0911,NaN,120311.0,2010-09-13,2012-04-17,NaN
4,ABM00000000000000011,SYNCID0000000034,323.70,debit,True,CA,NaN,other,2022-11-14,13:24:45,...,NaN,CA,ON,LONDON,0911,NaN,120311.0,2010-09-13,2012-04-17,NaN


In [37]:
final_data.shape

(1494097, 22)

In [38]:
# Rename columns
final_data = final_data.rename(columns={
    'country_x': 'trans_country',
    'province_x': 'trans_province',
    'city_x': 'trans_city',
    'country_y': 'cust_country',
    'province_y': 'cust_province',
    'city_y': 'cust_city'
})

In [39]:
final_data.head()

,transaction_id,customer_id,amount_cad,debit_credit,cash_indicator,trans_country,trans_province,trans_city,transaction_date,transaction_time,...,ecommerce_ind,cust_country,cust_province,cust_city,industry_code,employee_count,sales,established_date,onboard_date,industry
0,ABM00000000000000006,SYNCID0000000014,25.41,credit,False,CA,SK,REGINA,2022-11-16,17:37:41,...,NaN,CA,ON,MARKHAM,4275,0.0,NaN,2016-09-14,NaN,Painting and Decorating Work
1,ABM00000000000000008,SYNCID0000000034,238.17,debit,True,CA,ON,TORONTO,2022-11-18,10:22:59,...,NaN,CA,ON,LONDON,0911,NaN,120311.0,2010-09-13,2012-04-17,NaN
2,ABM00000000000000009,SYNCID0000000034,1655.43,credit,False,CA,NaN,other,2022-12-29,11:56:08,...,NaN,CA,ON,LONDON,0911,NaN,120311.0,2010-09-13,2012-04-17,NaN
3,ABM00000000000000010,SYNCID0000000034,620.69,credit,True,CA,NaN,other,2023-01-22,16:48:12,...,NaN,CA,ON,LONDON,0911,NaN,120311.0,2010-09-13,2012-04-17,NaN
4,ABM00000000000000011,SYNCID0000000034,323.70,debit,True,CA,NaN,other,2022-11-14,13:24:45,...,NaN,CA,ON,LONDON,0911,NaN,120311.0,2010-09-13,2012-04-17,NaN


In [40]:
# Save the final dataset
final_data.to_csv("final_transactions.csv", index=False)

## Create Features

### Load DF

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("final_transactions.csv")

<ipython-input-4-496edb7c48f7>:1: DtypeWarning: Columns (4,5,6,7,9,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("final_transactions.csv")


In [5]:
df.shape

(1494097, 22)

In [ ]:
df.info()

###  Transaction Frequency & Timing Features

In [6]:
# Ensure transaction_date and transaction_time are in datetime format
df["transaction_date"] = pd.to_datetime(df["transaction_date"])
df["transaction_time"] = pd.to_datetime(df["transaction_time"], format="%H:%M:%S", errors="coerce")

In [7]:
# Group transactions by customer
customer_group = df.groupby("customer_id")

# Compute average transactions per time period
df_freq = customer_group["transaction_id"].count().reset_index()
df_freq.rename(columns={"transaction_id": "txn_count"}, inplace=True)
df_freq["avg_txn_per_month"] = df_freq["txn_count"] / df["transaction_date"].dt.to_period("M").nunique()
df_freq["avg_txn_per_week"] = df_freq["txn_count"] / df["transaction_date"].dt.to_period("W").nunique()
df_freq["avg_txn_per_year"] = df_freq["txn_count"] / df["transaction_date"].dt.to_period("Y").nunique()

In [8]:
# Extract transaction hour and weekday
df["txn_hour"] = df["transaction_time"].dt.hour
df["txn_weekday"] = df["transaction_date"].dt.weekday  # 0=Monday, ..., 6=Sunday

# Flag weekend transactions (Saturday=5, Sunday=6)
df["weekend_txn"] = df["txn_weekday"].apply(lambda x: 1 if x in [5, 6] else 0)

# Flag odd-hour transactions (00:00-05:00)
df["odd_hour_txn"] = df["txn_hour"].apply(lambda x: 1 if 0 <= x <= 5 else 0)

In [9]:
# Merge frequency features back into the main dataset
df = df.merge(df_freq, on="customer_id", how="left")

# Display the updated dataset
df.head()

,transaction_id,customer_id,amount_cad,debit_credit,cash_indicator,trans_country,trans_province,trans_city,transaction_date,transaction_time,...,onboard_date,industry,txn_hour,txn_weekday,weekend_txn,odd_hour_txn,txn_count,avg_txn_per_month,avg_txn_per_week,avg_txn_per_year
0,ABM00000000000000006,SYNCID0000000014,25.41,credit,False,CA,SK,REGINA,2022-11-16,1900-01-01 17:37:41,...,NaN,Painting and Decorating Work,17.0,2,0,0,81,27.000000,5.785714,40.5
1,ABM00000000000000008,SYNCID0000000034,238.17,debit,True,CA,ON,TORONTO,2022-11-18,1900-01-01 10:22:59,...,2012-04-17,NaN,10.0,4,0,0,41,13.666667,2.928571,20.5
2,ABM00000000000000009,SYNCID0000000034,1655.43,credit,False,CA,NaN,other,2022-12-29,1900-01-01 11:56:08,...,2012-04-17,NaN,11.0,3,0,0,41,13.666667,2.928571,20.5
3,ABM00000000000000010,SYNCID0000000034,620.69,credit,True,CA,NaN,other,2023-01-22,1900-01-01 16:48:12,...,2012-04-17,NaN,16.0,6,1,0,41,13.666667,2.928571,20.5
4,ABM00000000000000011,SYNCID0000000034,323.70,debit,True,CA,NaN,other,2022-11-14,1900-01-01 13:24:45,...,2012-04-17,NaN,13.0,0,0,0,41,13.666667,2.928571,20.5


In [10]:
# Sort transactions by customer_id and transaction_date
df = df.sort_values(by=["customer_id", "transaction_date"])
df.head()

,transaction_id,customer_id,amount_cad,debit_credit,cash_indicator,trans_country,trans_province,trans_city,transaction_date,transaction_time,...,onboard_date,industry,txn_hour,txn_weekday,weekend_txn,odd_hour_txn,txn_count,avg_txn_per_month,avg_txn_per_week,avg_txn_per_year
576613,CHE00000000000000000,SYNCID0000000000,415.24,debit,NaN,NaN,NaN,NaN,2022-11-08,NaT,...,2022-05-02,"Estate, Trust and Agency Funds",NaN,1,0,0,3,1.000000,0.214286,1.5
1489339,WIR00000000000000000,SYNCID0000000000,6316.04,debit,NaN,NaN,NaN,NaN,2022-11-18,1900-01-01 00:00:00,...,2022-05-02,"Estate, Trust and Agency Funds",0.0,4,0,1,3,1.000000,0.214286,1.5
1489340,WIR00000000000000001,SYNCID0000000000,0.03,credit,NaN,NaN,NaN,NaN,2022-12-31,1900-01-01 00:00:00,...,2022-05-02,"Estate, Trust and Agency Funds",0.0,5,1,1,3,1.000000,0.214286,1.5
20658,CON00000000000000001,SYNCID0000000001,1.72,debit,NaN,NaN,NaN,other,2022-11-02,1900-01-01 13:28:57,...,2009-08-21,NaN,13.0,2,0,0,4,1.333333,0.285714,2.0
20660,CON00000000000000003,SYNCID0000000001,213.40,debit,NaN,CA,NB,MONCTON,2022-11-21,1900-01-01 09:15:48,...,2009-08-21,NaN,9.0,0,0,0,4,1.333333,0.285714,2.0


In [11]:
# Compute time gaps between consecutive transactions for each customer
df["txn_gap"] = df.groupby("customer_id")["transaction_date"].diff().dt.days

# Aggregation per customer
customer_agg = df.groupby("customer_id").agg(
    avg_gap_between_txns_month=("txn_gap", "mean"),  # Avg gap between transactions
    std_gap_between_txns=("txn_gap", "std"),  # Standard deviation of gap
    max_time_between_txns=("txn_gap", "max"),  # Max time gap
    min_time_between_txns=("txn_gap", "min"),  # Min time gap
)

# Compute average & std for same-day multiple transactions
same_day_txns = df.groupby(["customer_id", "transaction_date"])["transaction_id"].count().reset_index()
same_day_agg = same_day_txns.groupby("customer_id")["transaction_id"].agg(
    same_day_multiple_txn="mean",
    std_same_day_txn="std"
)

# Compute modality of transactions per month (how often a customer has the same txn count per month)
monthly_txn_counts = df.groupby(["customer_id", df["transaction_date"].dt.to_period("M")])["transaction_id"].count()
modality_txn_count = monthly_txn_counts.groupby("customer_id").apply(lambda x: (x.value_counts() > 1).sum()).rename("modality_txn_count")

# Merge all features into final dataset
df = df.merge(customer_agg, on="customer_id", how="left")
df = df.merge(same_day_agg, on="customer_id", how="left")
df = df.merge(modality_txn_count, on="customer_id", how="left")

In [12]:
# Replace NaNs with default values
df.fillna({
    "modality_txn_count": 0,
    "same_day_multiple_txn": 0,
    "std_same_day_txn": 0
}, inplace=True)

In [13]:
# Display final dataset
df.head()

,transaction_id,customer_id,amount_cad,debit_credit,cash_indicator,trans_country,trans_province,trans_city,transaction_date,transaction_time,...,avg_txn_per_week,avg_txn_per_year,txn_gap,avg_gap_between_txns_month,std_gap_between_txns,max_time_between_txns,min_time_between_txns,same_day_multiple_txn,std_same_day_txn,modality_txn_count
0,CHE00000000000000000,SYNCID0000000000,415.24,debit,NaN,NaN,NaN,NaN,2022-11-08,NaT,...,0.214286,1.5,NaN,26.5,23.334524,43.0,10.0,1.0,0.0,0
1,WIR00000000000000000,SYNCID0000000000,6316.04,debit,NaN,NaN,NaN,NaN,2022-11-18,1900-01-01 00:00:00,...,0.214286,1.5,10.0,26.5,23.334524,43.0,10.0,1.0,0.0,0
2,WIR00000000000000001,SYNCID0000000000,0.03,credit,NaN,NaN,NaN,NaN,2022-12-31,1900-01-01 00:00:00,...,0.214286,1.5,43.0,26.5,23.334524,43.0,10.0,1.0,0.0,0
3,CON00000000000000001,SYNCID0000000001,1.72,debit,NaN,NaN,NaN,other,2022-11-02,1900-01-01 13:28:57,...,0.285714,2.0,NaN,16.0,7.000000,21.0,8.0,1.0,0.0,0
4,CON00000000000000003,SYNCID0000000001,213.40,debit,NaN,CA,NB,MONCTON,2022-11-21,1900-01-01 09:15:48,...,0.285714,2.0,19.0,16.0,7.000000,21.0,8.0,1.0,0.0,0


### Transaction Amount & Distribution Features

In [14]:
import numpy as np

In [15]:
# Define percentiles for outlier detection
high_percentile = df["amount_cad"].quantile(0.99)
low_percentile = df["amount_cad"].quantile(0.01)

# Compute customer-level transaction stats
customer_agg = df.groupby("customer_id").agg(
    total_spent=("amount_cad", "sum"),  # Total amount spent per customer
    avg_spent=("amount_cad", "mean"),  # Average transaction amount per customer
    std_txn_amount_per_month=("amount_cad", "std"),  # Std deviation of transaction amounts
)

# Compute monthly transaction stats
monthly_agg = df.groupby(["customer_id", df["transaction_date"].dt.to_period("M")])["amount_cad"].agg(["mean", "std"])
monthly_agg = monthly_agg.groupby("customer_id").agg(
    avg_spent_per_month=("mean", "mean"),
    std_spent_per_month=("std", "mean")
)

# Compute deposit-withdrawal ratio
deposit_withdrawals = df.groupby("customer_id")["debit_credit"].apply(lambda x: (x == 1).sum() / (x == -1).sum() if (x == -1).sum() > 0 else np.nan)
deposit_withdrawals = deposit_withdrawals.rename("deposit_withdrawal_ratio")

# Outlier detection
df["high_txn_outlier"] = (df["amount_cad"] > high_percentile).astype(int)
df["low_txn_outlier"] = (df["amount_cad"] < low_percentile).astype(int)

# Flag large cash transactions
df["large_cash_txn"] = ((df["cash_indicator"] == 1) & (df["amount_cad"] > high_percentile)).astype(int)

# Compute percentage of round transactions
df["is_round_txn"] = df["amount_cad"].apply(lambda x: 1 if x % 10 == 0 else 0)
round_txn_agg = df.groupby("customer_id")["is_round_txn"].mean() * 100
round_txn_agg = round_txn_agg.rename("%_round_txns")

In [16]:
# Merge all computed features
df = df.merge(customer_agg, on="customer_id", how="left")
df = df.merge(monthly_agg, on="customer_id", how="left")
df = df.merge(deposit_withdrawals, on="customer_id", how="left")
df = df.merge(round_txn_agg, on="customer_id", how="left")

In [17]:
# Replace NaN values with defaults
df.fillna({
    "std_txn_amount_per_month": 0,
    "std_spent_per_month": 0,
    "deposit_withdrawal_ratio": 0,
    "%_round_txns": 0
}, inplace=True)

In [18]:
df.head()

,transaction_id,customer_id,amount_cad,debit_credit,cash_indicator,trans_country,trans_province,trans_city,transaction_date,transaction_time,...,low_txn_outlier,large_cash_txn,is_round_txn,total_spent,avg_spent,std_txn_amount_per_month,avg_spent_per_month,std_spent_per_month,deposit_withdrawal_ratio,%_round_txns
0,CHE00000000000000000,SYNCID0000000000,415.24,debit,NaN,NaN,NaN,NaN,2022-11-08,NaT,...,0,0,0,6731.31,2243.7700,3532.79451,1682.835,4172.495694,0.0,0.0
1,WIR00000000000000000,SYNCID0000000000,6316.04,debit,NaN,NaN,NaN,NaN,2022-11-18,1900-01-01 00:00:00,...,0,0,0,6731.31,2243.7700,3532.79451,1682.835,4172.495694,0.0,0.0
2,WIR00000000000000001,SYNCID0000000000,0.03,credit,NaN,NaN,NaN,NaN,2022-12-31,1900-01-01 00:00:00,...,0,0,0,6731.31,2243.7700,3532.79451,1682.835,4172.495694,0.0,0.0
3,CON00000000000000001,SYNCID0000000001,1.72,debit,NaN,NaN,NaN,other,2022-11-02,1900-01-01 13:28:57,...,0,0,0,291.39,72.8475,96.96497,68.665,118.314616,0.0,0.0
4,CON00000000000000003,SYNCID0000000001,213.40,debit,NaN,CA,NB,MONCTON,2022-11-21,1900-01-01 09:15:48,...,0,0,0,291.39,72.8475,96.96497,68.665,118.314616,0.0,0.0


In [19]:
df.shape

(1494097, 49)

### Transaction Categorization & Size-Based Analysis

In [20]:
# Compute percentiles for small and medium transaction thresholds
SMALL_THRESHOLD = np.percentile(df["amount_cad"], 33)  # 33rd percentile = small threshold
MEDIUM_THRESHOLD = np.percentile(df["amount_cad"], 66)  # 66th percentile = medium threshold

print(f"Computed Small Transaction Threshold: {SMALL_THRESHOLD}")
print(f"Computed Medium Transaction Threshold: {MEDIUM_THRESHOLD}")

Computed Small Transaction Threshold: 94.61
Computed Medium Transaction Threshold: 607.0


In [21]:
# Categorize transactions into small, medium, large
df["txn_category"] = pd.cut(
    df["amount_cad"],
    bins=[-np.inf, SMALL_THRESHOLD, MEDIUM_THRESHOLD, np.inf],
    labels=["small", "medium", "large"]
)

# Count number of transactions per category for each customer
txn_counts = df.groupby(["customer_id", "txn_category"])["transaction_id"].count().unstack(fill_value=0)
txn_counts.columns = [f"{col}_txn_count" for col in txn_counts.columns]  # Rename columns

# Compute days between transactions for each category
df["transaction_date"] = pd.to_datetime(df["transaction_date"])
df = df.sort_values(by=["customer_id", "transaction_date"])
df["txn_gap"] = df.groupby("customer_id")["transaction_date"].diff().dt.days

# Compute avg days between transactions per category
txn_gaps = df.groupby(["customer_id", "txn_category"])["txn_gap"].mean().unstack(fill_value=0)
txn_gaps.columns = [f"avg_days_between_{col}_txn" for col in txn_gaps.columns]  # Rename columns

<ipython-input-21-45fbdee50759>:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  txn_counts = df.groupby(["customer_id", "txn_category"])["transaction_id"].count().unstack(fill_value=0)
<ipython-input-21-45fbdee50759>:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  txn_gaps = df.groupby(["customer_id", "txn_category"])["txn_gap"].mean().unstack(fill_value=0)


In [22]:
# Merge all computed features
df = df.merge(txn_counts, on="customer_id", how="left")
df = df.merge(txn_gaps, on="customer_id", how="left")

In [23]:
# Replace NaN values with default
df.fillna({
    "small_txn_count": 0,
    "medium_txn_count": 0,
    "large_txn_count": 0,
    "avg_days_between_large_txn": 0,
    "avg_days_between_medium_txn": 0,
    "avg_days_between_small_txn": 0
}, inplace=True)

In [24]:
df.head()

,transaction_id,customer_id,amount_cad,debit_credit,cash_indicator,trans_country,trans_province,trans_city,transaction_date,transaction_time,...,std_spent_per_month,deposit_withdrawal_ratio,%_round_txns,txn_category,small_txn_count,medium_txn_count,large_txn_count,avg_days_between_small_txn,avg_days_between_medium_txn,avg_days_between_large_txn
0,CHE00000000000000000,SYNCID0000000000,415.24,debit,NaN,NaN,NaN,NaN,2022-11-08,NaT,...,4172.495694,0.0,0.0,medium,1,1,1,43.0,0.0,10.0
1,WIR00000000000000000,SYNCID0000000000,6316.04,debit,NaN,NaN,NaN,NaN,2022-11-18,1900-01-01 00:00:00,...,4172.495694,0.0,0.0,large,1,1,1,43.0,0.0,10.0
2,WIR00000000000000001,SYNCID0000000000,0.03,credit,NaN,NaN,NaN,NaN,2022-12-31,1900-01-01 00:00:00,...,4172.495694,0.0,0.0,small,1,1,1,43.0,0.0,10.0
3,CON00000000000000001,SYNCID0000000001,1.72,debit,NaN,NaN,NaN,other,2022-11-02,1900-01-01 13:28:57,...,118.314616,0.0,0.0,small,3,1,0,14.5,19.0,0.0
4,CON00000000000000003,SYNCID0000000001,213.40,debit,NaN,CA,NB,MONCTON,2022-11-21,1900-01-01 09:15:48,...,118.314616,0.0,0.0,medium,3,1,0,14.5,19.0,0.0


In [25]:
df.shape

(1494097, 56)

### Ratio & Aggregation-Based Features

In [26]:
# Ensure transaction_date is in datetime format
df["transaction_date"] = pd.to_datetime(df["transaction_date"])

In [27]:
# Compute the last 60 days for sudden spike ratio calculation
last_60_days = df["transaction_date"].max() - pd.DateOffset(days=60)

# Compute transaction count in the last 90 days and historic average
recent_txn_count = df[df["transaction_date"] >= last_60_days].groupby("customer_id")["transaction_id"].count()
historic_txn_count = df.groupby("customer_id")["transaction_id"].count()

# Compute sudden spike ratio (recent volume / historic avg)
sudden_spike_ratio = (recent_txn_count / historic_txn_count).fillna(0).rename("sudden_spike_ratio")

In [28]:
# Compute in-out balance (sum of positive transactions - sum of negative transactions)
in_out_balance = df.groupby("customer_id")["amount_cad"].apply(lambda x: x[x > 0].sum() - abs(x[x < 0].sum())).rename("in_out_balance")

# Compute percentage of transactions in first & last 5 days of each month
df["day_of_month"] = df["transaction_date"].dt.day
df["first_5_days"] = df["day_of_month"].apply(lambda x: 1 if x <= 5 else 0)
df["last_5_days"] = df["day_of_month"].apply(lambda x: 1 if x >= 25 else 0)

first_5_txn_ratio = df.groupby("customer_id")["first_5_days"].mean() * 100
last_5_txn_ratio = df.groupby("customer_id")["last_5_days"].mean() * 100

first_5_txn_ratio = first_5_txn_ratio.rename("%txn_first_5_days_month")
last_5_txn_ratio = last_5_txn_ratio.rename("%txn_last_5_days_month")


In [29]:
# Compute percentage of months with sudden standard deviation change
monthly_std = df.groupby(["customer_id", df["transaction_date"].dt.to_period("M")])["amount_cad"].std()
monthly_std_change = monthly_std.groupby("customer_id").apply(lambda x: ((x.pct_change().abs() > 0.5).sum() / len(x)) * 100).rename("%months_irregular_std")

<ipython-input-29-f6b0c234314b>:3: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  monthly_std_change = monthly_std.groupby("customer_id").apply(lambda x: ((x.pct_change().abs() > 0.5).sum() / len(x)) * 100).rename("%months_irregular_std")


In [30]:
# Merge computed features into final dataset
df = df.merge(sudden_spike_ratio, on="customer_id", how="left")
df = df.merge(in_out_balance, on="customer_id", how="left")
df = df.merge(first_5_txn_ratio, on="customer_id", how="left")
df = df.merge(last_5_txn_ratio, on="customer_id", how="left")
df = df.merge(monthly_std_change, on="customer_id", how="left")

In [31]:
# Replace NaN values with default
df.fillna({
    "sudden_spike_ratio": 0,
    "in_out_balance": 0,
    "%txn_first_5_days_month": 0,
    "%txn_last_5_days_month": 0,
    "%months_irregular_std": 0
}, inplace=True)

In [32]:
df.head()

,transaction_id,customer_id,amount_cad,debit_credit,cash_indicator,trans_country,trans_province,trans_city,transaction_date,transaction_time,...,avg_days_between_medium_txn,avg_days_between_large_txn,day_of_month,first_5_days,last_5_days,sudden_spike_ratio,in_out_balance,%txn_first_5_days_month,%txn_last_5_days_month,%months_irregular_std
0,CHE00000000000000000,SYNCID0000000000,415.24,debit,NaN,NaN,NaN,NaN,2022-11-08,NaT,...,0.0,10.0,8,0,0,0.333333,6731.31,0.0,33.333333,0.0
1,WIR00000000000000000,SYNCID0000000000,6316.04,debit,NaN,NaN,NaN,NaN,2022-11-18,1900-01-01 00:00:00,...,0.0,10.0,18,0,0,0.333333,6731.31,0.0,33.333333,0.0
2,WIR00000000000000001,SYNCID0000000000,0.03,credit,NaN,NaN,NaN,NaN,2022-12-31,1900-01-01 00:00:00,...,0.0,10.0,31,0,1,0.333333,6731.31,0.0,33.333333,0.0
3,CON00000000000000001,SYNCID0000000001,1.72,debit,NaN,NaN,NaN,other,2022-11-02,1900-01-01 13:28:57,...,19.0,0.0,2,1,0,0.250000,291.39,25.0,25.000000,0.0
4,CON00000000000000003,SYNCID0000000001,213.40,debit,NaN,CA,NB,MONCTON,2022-11-21,1900-01-01 09:15:48,...,19.0,0.0,21,0,0,0.250000,291.39,25.0,25.000000,0.0


In [33]:
df.shape

(1494097, 64)

### Merchant & Industry-Based Features

In [34]:
# Most frequent merchant category per customer
most_freq_merchant_category = df.groupby("customer_id")["merchant_category"].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else "Unknown")
most_freq_merchant_category = most_freq_merchant_category.rename("most_freq_merchant_category")

# Most frequent industry code per customer
most_freq_industry_code = df.groupby("customer_id")["industry_code"].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else "Unknown")
most_freq_industry_code = most_freq_industry_code.rename("most_freq_industry_code")

# Modality of transaction amount (how often the same amount repeats)
modality_txn_amount = df.groupby(["customer_id", "amount_cad"]).size().groupby("customer_id").apply(lambda x: (x > 1).sum()).rename("modality_txn_amount")

# Modality of transaction industry (how often transactions occur in the same industry)
modality_txn_industry = df.groupby(["customer_id", "industry_code"]).size().groupby("customer_id").apply(lambda x: (x > 1).sum()).rename("modality_txn_industry")

# Avg transaction amount per merchant type
avg_txn_amount_per_merchant_type = df.groupby(["customer_id", "merchant_category"])["amount_cad"].mean().groupby("customer_id").mean()
avg_txn_amount_per_merchant_type = avg_txn_amount_per_merchant_type.rename("avg_txn_amount_per_merchant_type")

In [35]:
# Merge all computed features
df = df.merge(most_freq_merchant_category, on="customer_id", how="left")
df = df.merge(most_freq_industry_code, on="customer_id", how="left")
df = df.merge(modality_txn_amount, on="customer_id", how="left")
df = df.merge(modality_txn_industry, on="customer_id", how="left")
df = df.merge(avg_txn_amount_per_merchant_type, on="customer_id", how="left")

In [36]:
# Replace NaN values with defaults
df.fillna({
    "most_freq_merchant_category": "Unknown",
    "most_freq_industry_code": "Unknown",
    "modality_txn_amount": 0,
    "modality_txn_industry": 0,
    "avg_txn_amount_per_merchant_type": 0
}, inplace=True)

In [37]:
df.head()

,transaction_id,customer_id,amount_cad,debit_credit,cash_indicator,trans_country,trans_province,trans_city,transaction_date,transaction_time,...,sudden_spike_ratio,in_out_balance,%txn_first_5_days_month,%txn_last_5_days_month,%months_irregular_std,most_freq_merchant_category,most_freq_industry_code,modality_txn_amount,modality_txn_industry,avg_txn_amount_per_merchant_type
0,CHE00000000000000000,SYNCID0000000000,415.24,debit,NaN,NaN,NaN,NaN,2022-11-08,NaT,...,0.333333,6731.31,0.0,33.333333,0.0,Unknown,7292,0,1.0,0.000000
1,WIR00000000000000000,SYNCID0000000000,6316.04,debit,NaN,NaN,NaN,NaN,2022-11-18,1900-01-01 00:00:00,...,0.333333,6731.31,0.0,33.333333,0.0,Unknown,7292,0,1.0,0.000000
2,WIR00000000000000001,SYNCID0000000000,0.03,credit,NaN,NaN,NaN,NaN,2022-12-31,1900-01-01 00:00:00,...,0.333333,6731.31,0.0,33.333333,0.0,Unknown,7292,0,1.0,0.000000
3,CON00000000000000001,SYNCID0000000001,1.72,debit,NaN,NaN,NaN,other,2022-11-02,1900-01-01 13:28:57,...,0.250000,291.39,25.0,25.000000,0.0,other,Unknown,0,0.0,84.418333
4,CON00000000000000003,SYNCID0000000001,213.40,debit,NaN,CA,NB,MONCTON,2022-11-21,1900-01-01 09:15:48,...,0.250000,291.39,25.0,25.000000,0.0,other,Unknown,0,0.0,84.418333


### Location-Based Risk Features

Code to Identify High-Risk Countries Dynamically

In [38]:
# Step 1: Count outlier transactions per country, province, and city
country_risk_score = df.groupby("trans_country")["high_txn_outlier"].sum()
province_risk_score = df.groupby("trans_province")["high_txn_outlier"].sum()
city_risk_score = df.groupby("trans_city")["high_txn_outlier"].sum()

# Step 2: Compute dynamic threshold using 95th percentile for high-risk classification
X_country = np.percentile(country_risk_score, 90)
X_province = np.percentile(province_risk_score, 90)
X_city = np.percentile(city_risk_score, 95)

# Step 3: Identify high-risk countries, provinces, and cities
high_risk_countries = country_risk_score[country_risk_score >= X_country].index.tolist()
high_risk_provinces = province_risk_score[province_risk_score >= X_province].index.tolist()
high_risk_cities = city_risk_score[city_risk_score >= X_city].index.tolist()

print(f"Computed X threshold: {X_country, X_province, X_city}")
print(f"Computed High-Risk Countries: {high_risk_countries}")
print(f"Computed High-Risk Provinces: {high_risk_provinces}")
print(f"Computed High-Risk Cities: {high_risk_cities}")

Computed X threshold: (86.0, 11.0, 5.0)
Computed High-Risk Countries: ['CA']
Computed High-Risk Provinces: ['AB', 'NS', 'ON']
Computed High-Risk Cities: ['BRAMPTON', 'CALGARY', 'EDMONTON', 'LONDON', 'MISSISSAUGA', 'OTTAWA', 'TORONTO', 'WOODBRIDGE', 'other']


In [39]:
# Step 4: Assign high-risk labels to each transaction
df["txn_high_risk_country"] = df["trans_country"].apply(lambda x: 1 if x in high_risk_countries else 0)
df["txn_high_risk_province"] = df["trans_province"].apply(lambda x: 1 if x in high_risk_provinces else 0)
df["txn_high_risk_city"] = df["trans_city"].apply(lambda x: 1 if x in high_risk_cities else 0)

# Display the first few rows
df[["customer_id", "trans_country", "trans_province", "trans_city", "txn_high_risk_country", "txn_high_risk_province", "txn_high_risk_city"]].head()

,customer_id,trans_country,trans_province,trans_city,txn_high_risk_country,txn_high_risk_province,txn_high_risk_city
0,SYNCID0000000000,NaN,NaN,NaN,0,0,0
1,SYNCID0000000000,NaN,NaN,NaN,0,0,0
2,SYNCID0000000000,NaN,NaN,NaN,0,0,0
3,SYNCID0000000001,NaN,NaN,other,0,0,1
4,SYNCID0000000001,CA,NB,MONCTON,1,0,0


In [40]:
# Count outlier transactions per country, province, city
country_risk = df[df["high_txn_outlier"] == 1].groupby("trans_country")["transaction_id"].count().rename("country_risk")
province_risk = df[df["high_txn_outlier"] == 1].groupby("trans_province")["transaction_id"].count().rename("province_risk")
city_risk = df[df["high_txn_outlier"] == 1].groupby("trans_city")["transaction_id"].count().rename("city_risk")

In [41]:
# Compute customer's home country from KYC data
df["outside_home_country"] = (df["trans_country"] != df["cust_country"]).astype(int)

# Compute % of transactions outside user’s home country
txn_outside_user_country = df.groupby("customer_id")["outside_home_country"].mean() * 100
txn_outside_user_country = txn_outside_user_country.rename("%txn_outside_user_country")

# Compute % of transactions to high-risk countries, provinces, and cities
df["txn_high_risk_country"] = df["trans_country"].apply(lambda x: 1 if x in high_risk_countries else 0)
df["txn_high_risk_province"] = df["trans_province"].apply(lambda x: 1 if x in high_risk_provinces else 0)
df["txn_high_risk_city"] = df["trans_city"].apply(lambda x: 1 if x in high_risk_cities else 0)

txn_high_risk_countries = df.groupby("customer_id")["txn_high_risk_country"].mean() * 100
txn_high_risk_provinces = df.groupby("customer_id")["txn_high_risk_province"].mean() * 100
txn_high_risk_cities = df.groupby("customer_id")["txn_high_risk_city"].mean() * 100

txn_high_risk_countries = txn_high_risk_countries.rename("%txn_high_risk_countries")
txn_high_risk_provinces = txn_high_risk_provinces.rename("%txn_high_risk_provinces")
txn_high_risk_cities = txn_high_risk_cities.rename("%txn_high_risk_cities")

In [42]:
# Compute # of unique countries transactions were sent to
num_unique_countries = df.groupby("customer_id")["trans_country"].nunique().rename("num_unique_countries")

Code to identify high risk industries dynamically

In [43]:
# Step 1: Count outlier transactions per industry
industry_risk_score = df.groupby("industry_code")["high_txn_outlier"].sum()

# Step 2: Compute dynamic threshold for high-risk classification
X_industry = np.percentile(industry_risk_score, 90)  # Select industries in the top 10% risk score
high_risk_industries = industry_risk_score[industry_risk_score >= X_industry].index.tolist()

print(f"Computed X threshold for industries: {X_industry}")
print(f"High-risk industries: {high_risk_industries}")

Computed X threshold for industries: 87.80000000000001
High-risk industries: ['0112', '4013', '4261', '4561', '6031', '6921', '7215', '7292', '7511', '7611', '7721', '7731', '7739', '7752', '7759', '7761', '7771', '7799', '8651', '8652', '8653', '9211', '9712', '9811', '9861', 'other']


In [44]:
# Compute % of transactions to high-risk industries
df["high_risk_industry"] = df["industry_code"].apply(lambda x: 1 if x in high_risk_industries else 0)
txn_high_risk_industries = df.groupby("customer_id")["high_risk_industry"].mean() * 100
txn_high_risk_industries = txn_high_risk_industries.rename("%txn_high_risk_industries")

In [45]:
# Merge computed features into the dataset
df = df.merge(country_risk, on="trans_country", how="left")
df = df.merge(province_risk, on="trans_province", how="left")
df = df.merge(city_risk, on="trans_city", how="left")

df = df.merge(txn_outside_user_country, on="customer_id", how="left")
df = df.merge(txn_high_risk_countries, on="customer_id", how="left")
df = df.merge(txn_high_risk_provinces, on="customer_id", how="left")
df = df.merge(txn_high_risk_cities, on="customer_id", how="left")
df = df.merge(txn_high_risk_industries, on="customer_id", how="left")
df = df.merge(num_unique_countries, on="customer_id", how="left")

In [46]:
# Replace NaN values with default
df.fillna({
    "country_risk": 0,
    "province_risk": 0,
    "city_risk": 0,
    "%txn_outside_user_country": 0,
    "%txn_high_risk_countries": 0,
    "%txn_high_risk_provinces": 0,
    "%txn_high_risk_cities": 0,
    "%txn_high_risk_industries": 0,
    "num_unique_countries": 0
}, inplace=True)

In [47]:
df.head()

,transaction_id,customer_id,amount_cad,debit_credit,cash_indicator,trans_country,trans_province,trans_city,transaction_date,transaction_time,...,high_risk_industry,country_risk,province_risk,city_risk,%txn_outside_user_country,%txn_high_risk_countries,%txn_high_risk_provinces,%txn_high_risk_cities,%txn_high_risk_industries,num_unique_countries
0,CHE00000000000000000,SYNCID0000000000,415.24,debit,NaN,NaN,NaN,NaN,2022-11-08,NaT,...,1,0.0,0.0,0.0,100.0,0.0,0.0,0.0,100.0,0
1,WIR00000000000000000,SYNCID0000000000,6316.04,debit,NaN,NaN,NaN,NaN,2022-11-18,1900-01-01 00:00:00,...,1,0.0,0.0,0.0,100.0,0.0,0.0,0.0,100.0,0
2,WIR00000000000000001,SYNCID0000000000,0.03,credit,NaN,NaN,NaN,NaN,2022-12-31,1900-01-01 00:00:00,...,1,0.0,0.0,0.0,100.0,0.0,0.0,0.0,100.0,0
3,CON00000000000000001,SYNCID0000000001,1.72,debit,NaN,NaN,NaN,other,2022-11-02,1900-01-01 13:28:57,...,0,0.0,0.0,93.0,50.0,50.0,25.0,75.0,0.0,1
4,CON00000000000000003,SYNCID0000000001,213.40,debit,NaN,CA,NB,MONCTON,2022-11-21,1900-01-01 09:15:48,...,0,164.0,2.0,1.0,50.0,50.0,25.0,75.0,0.0,1


In [48]:
df.shape

(1494097, 83)

### Transaction Timing Trends

In [49]:
# Ensure transaction_time is in datetime format
df["transaction_time"] = pd.to_datetime(df["transaction_time"], format="%H:%M:%S", errors="coerce")

In [50]:
# Extract transaction hour
df["txn_hour"] = df["transaction_time"].dt.hour

# Flag night transactions (00:00 - 06:00) and day transactions (06:00 - 18:00)
df["night_txn"] = df["txn_hour"].apply(lambda x: 1 if 0 <= x < 6 else 0)
df["day_txn"] = df["txn_hour"].apply(lambda x: 1 if 6 <= x < 18 else 0)

# Compute % of night transactions per customer
night_txn_ratio = df.groupby("customer_id")["night_txn"].mean() * 100
night_txn_ratio = night_txn_ratio.rename("%night_txns")

# Compute % of day transactions per customer
day_txn_ratio = df.groupby("customer_id")["day_txn"].mean() * 100
day_txn_ratio = day_txn_ratio.rename("%day_txns")

# Compute standard deviation of night transactions per month
monthly_night_txn = df.groupby(["customer_id", df["transaction_time"].dt.to_period("M")])["night_txn"].mean()
std_night_txn_per_month = monthly_night_txn.groupby("customer_id").std().rename("std_night_txns")

In [51]:
# Merge computed features into the dataset
df = df.merge(night_txn_ratio, on="customer_id", how="left")
df = df.merge(day_txn_ratio, on="customer_id", how="left")
df = df.merge(std_night_txn_per_month, on="customer_id", how="left")

In [52]:
# Replace NaN values with default
df.fillna({
    "%night_txns": 0,
    "%day_txns": 0,
    "std_night_txns": 0
}, inplace=True)

In [53]:
df.head()

,transaction_id,customer_id,amount_cad,debit_credit,cash_indicator,trans_country,trans_province,trans_city,transaction_date,transaction_time,...,%txn_high_risk_countries,%txn_high_risk_provinces,%txn_high_risk_cities,%txn_high_risk_industries,num_unique_countries,night_txn,day_txn,%night_txns,%day_txns,std_night_txns
0,CHE00000000000000000,SYNCID0000000000,415.24,debit,NaN,NaN,NaN,NaN,2022-11-08,NaT,...,0.0,0.0,0.0,100.0,0,0,0,66.666667,0.0,0.0
1,WIR00000000000000000,SYNCID0000000000,6316.04,debit,NaN,NaN,NaN,NaN,2022-11-18,1900-01-01 00:00:00,...,0.0,0.0,0.0,100.0,0,1,0,66.666667,0.0,0.0
2,WIR00000000000000001,SYNCID0000000000,0.03,credit,NaN,NaN,NaN,NaN,2022-12-31,1900-01-01 00:00:00,...,0.0,0.0,0.0,100.0,0,1,0,66.666667,0.0,0.0
3,CON00000000000000001,SYNCID0000000001,1.72,debit,NaN,NaN,NaN,other,2022-11-02,1900-01-01 13:28:57,...,50.0,25.0,75.0,0.0,1,0,1,0.000000,100.0,0.0
4,CON00000000000000003,SYNCID0000000001,213.40,debit,NaN,CA,NB,MONCTON,2022-11-21,1900-01-01 09:15:48,...,50.0,25.0,75.0,0.0,1,0,1,0.000000,100.0,0.0


In [54]:
df.shape

(1494097, 88)

### Employee & Business Scaling Adjustments

In [55]:
# Ensure employee_count is not zero to avoid division errors
df["employee_count"] = df["employee_count"].replace(0, np.nan)  # Replace 0 with NaN to avoid division by zero

# Compute total transactions per customer
txn_count = df.groupby("customer_id")["transaction_id"].count().rename("txn_count")

# Compute total transaction amount per customer
total_spent = df.groupby("customer_id")["amount_cad"].sum().rename("total_spent")

# Compute transactions per employee
txn_per_employee = (txn_count / df.groupby("customer_id")["employee_count"].first()).rename("txn_per_employee")

# Compute average transaction amount per employee
avg_txn_amount_per_employee = (total_spent / df.groupby("customer_id")["employee_count"].first()).rename("avg_txn_amount_per_employee")

In [56]:
# Merge computed features into the dataset
df = df.merge(txn_per_employee, on="customer_id", how="left")
df = df.merge(avg_txn_amount_per_employee, on="customer_id", how="left")

In [57]:
# Replace NaN values with default
df.fillna({
    "txn_per_employee": 0,
    "avg_txn_amount_per_employee": 0
}, inplace=True)

In [58]:
df.head()

,transaction_id,customer_id,amount_cad,debit_credit,cash_indicator,trans_country,trans_province,trans_city,transaction_date,transaction_time,...,%txn_high_risk_cities,%txn_high_risk_industries,num_unique_countries,night_txn,day_txn,%night_txns,%day_txns,std_night_txns,txn_per_employee,avg_txn_amount_per_employee
0,CHE00000000000000000,SYNCID0000000000,415.24,debit,NaN,NaN,NaN,NaN,2022-11-08,NaT,...,0.0,100.0,0,0,0,66.666667,0.0,0.0,0.0,0.0
1,WIR00000000000000000,SYNCID0000000000,6316.04,debit,NaN,NaN,NaN,NaN,2022-11-18,1900-01-01 00:00:00,...,0.0,100.0,0,1,0,66.666667,0.0,0.0,0.0,0.0
2,WIR00000000000000001,SYNCID0000000000,0.03,credit,NaN,NaN,NaN,NaN,2022-12-31,1900-01-01 00:00:00,...,0.0,100.0,0,1,0,66.666667,0.0,0.0,0.0,0.0
3,CON00000000000000001,SYNCID0000000001,1.72,debit,NaN,NaN,NaN,other,2022-11-02,1900-01-01 13:28:57,...,75.0,0.0,1,0,1,0.000000,100.0,0.0,0.0,0.0
4,CON00000000000000003,SYNCID0000000001,213.40,debit,NaN,CA,NB,MONCTON,2022-11-21,1900-01-01 09:15:48,...,75.0,0.0,1,0,1,0.000000,100.0,0.0,0.0,0.0


### Outlier & Anomaly Detection

In [73]:
# Compute percentiles for outlier detection
high_percentile = np.percentile(df["amount_cad"], 99)
low_percentile = np.percentile(df["amount_cad"], 3)

In [74]:
df["amount_cad"].max(), df["amount_cad"].min()

(17809059.15, -97063.38)

In [75]:
high_percentile, low_percentile

(51575.869200000256, 2.01)

In [76]:
# Flag high and low transaction outliers
df["high_txn_outlier"] = (df["amount_cad"] > high_percentile).astype(int)
df["low_txn_outlier"] = (df["amount_cad"] < low_percentile).astype(int)

# Flag large cash transactions
df["large_cash_txn"] = ((df["cash_indicator"] == 1) & (df["amount_cad"] > high_percentile)).astype(int)

In [77]:
# Ensure transaction_time is in datetime format
df["transaction_time"] = pd.to_datetime(df["transaction_time"], format="%H:%M:%S", errors="coerce")
df["txn_hour"] = df["transaction_time"].dt.hour

# Flag odd-hour transactions (00:00 - 05:00)
df["odd_hour_txn"] = df["txn_hour"].apply(lambda x: 1 if 0 <= x < 5 else 0)

# Compute % of night transactions (00:00 - 06:00) and day transactions (06:00 - 18:00)
df["night_txn"] = df["txn_hour"].apply(lambda x: 1 if 0 <= x < 6 else 0)
df["day_txn"] = df["txn_hour"].apply(lambda x: 1 if 6 <= x < 18 else 0)

night_txn_ratio = df.groupby("customer_id")["night_txn"].mean() * 100
night_txn_ratio = night_txn_ratio.rename("%night_txns")

day_txn_ratio = df.groupby("customer_id")["day_txn"].mean() * 100
day_txn_ratio = day_txn_ratio.rename("%day_txns")

# Compute standard deviation of % night transactions per month
monthly_night_txn = df.groupby(["customer_id", df["transaction_time"].dt.to_period("M")])["night_txn"].mean()
std_night_txn_per_month = monthly_night_txn.groupby("customer_id").std().rename("std_night_txns")

# Compute standard deviation of % day transactions per month
monthly_day_txn = df.groupby(["customer_id", df["transaction_time"].dt.to_period("M")])["day_txn"].mean()
std_day_txn_per_month = monthly_day_txn.groupby("customer_id").std().rename("std_day_txns")

# Compute % of transactions that are round amounts
df["round_txn_extra"] = df["amount_cad"].apply(lambda x: 1 if x % 100 == 0 else 0)
round_txn_extra_ratio = df.groupby("customer_id")["round_txn_extra"].mean() * 100
round_txn_extra_ratio = round_txn_extra_ratio.rename("%_round_txns_extra")

In [78]:
# Merge computed features into the dataset
df = df.merge(night_txn_ratio, on="customer_id", how="left")
df = df.merge(day_txn_ratio, on="customer_id", how="left")
df = df.merge(std_night_txn_per_month, on="customer_id", how="left")
df = df.merge(std_day_txn_per_month, on="customer_id", how="left")
df = df.merge(round_txn_extra_ratio, on="customer_id", how="left")

In [79]:
# Replace NaN values with default
df.fillna({
    "%night_txns": 0,
    "%day_txns": 0,
    "std_night_txns": 0,
    "std_day_txns": 0,
    "%_round_txns_extra": 0
}, inplace=True)

### Now export the data

In [80]:
df.to_csv("imi_features.csv", index=False)

In [2]:
### import the data back to check if the shape is retained
import pandas as pd
features = pd.read_csv("imi_features.csv")

<ipython-input-2-16657b0a7439>:3: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  features = pd.read_csv("imi_features.csv")


In [3]:
features.shape

(1494097, 96)